In [20]:
import os
os.chdir('c:\\projects\\udacity\\CarND-Advanced-Lane-Lines\\')
sys.path.insert(1, os.getcwd())

In [31]:
%matplotlib notebook

import scipy.misc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from camera_img_undistorter import CameraImageUndistorter
from camera_birdview_transform import CameraImagePerspectiveTransform

In [28]:
def plot_images(img_src, img_dst):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img_src)
    ax1.set_title('Source Image', fontsize=50)
    ax2.imshow(img_dst, cmap='gray')
    ax2.set_title('Destination Image', fontsize=50)
    ax2.imshow(img_dst, cmap='gray') # TODO make a parameter
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

In [29]:
img_transformer = CameraImagePerspectiveTransform()
img_undistorter = CameraImageUndistorter()

In [32]:
img = cv2.imread('./test_images/test1.jpg')
img_undistorted = img_undistorter.undistort(img)
plot_images(img, img_undistorted)

<IPython.core.display.Javascript object>

<div id='6a908024-9a5d-441e-a359-911ce89babfb'></div>

<IPython.core.display.Javascript object>

<div id='fc5dbb9a-7d29-45ac-b690-176da40cd371'></div>